In [1]:
pip install tensorflow

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB ? eta 0:00:00
   ---------------------------------------- 0.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/377.1 MB 10.2 MB/s eta 0:00:37
   ---------------------------------------- 1.1/377.1 MB 16.8 MB/s eta 0:00:23
   ---------------------------------------- 1.5/377.1 MB 10.5 MB/s eta 0:00:36
   ---------------------------------------- 1.8/377.1 MB 9.6 MB/s eta 0:00:40
   ---------------------------------------- 2.0/377.1 MB 8.5 MB/s eta 0:00:45
   ---------------------------------------- 2.2/377.1 MB 7.9 MB/s eta 0:00:48
   ---------------------------------------- 2.3/377.1 MB 7.7 MB/s eta 0:00:49
   ---------------------------------------- 2.6/377.1 MB 6.9 MB/s eta 0:00:54
   --


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ------------------------------ ------- 300.1/377.1 MB 816.3 kB/s eta 0:01:35
   ------------------------------ ------- 300.1/377.1 MB 814.3 kB/s eta 0:01:35
   ------------------------------ ------- 300.2/377.1 MB 811.2 kB/s eta 0:01:35
   ------------------------------ ------- 300.2/377.1 MB 810.3 kB/s eta 0:01:35
   ------------------------------ ------- 300.2/377.1 MB 808.2 kB/s eta 0:01:36
   ------------------------------ ------- 300.3/377.1 MB 807.3 kB/s eta 0:01:36
   ------------------------------ ------- 300.3/377.1 MB 806.3 kB/s eta 0:01:36
   ------------------------------ ------- 300.3/377.1 MB 802.3 kB/s eta 0:01:36
   ------------------------------ ------- 300.4/377.1 MB 802.3 kB/s eta 0:01:36
   ------------------------------ ------- 300.4/377.1 MB 800.3 kB/s eta 0:01:36
   ------------------------------ ------- 300.4/377.1 MB 798.4 kB/s eta 0:01:37
   ------------------------------ ------- 300.4/377.1 MB 797.5 kB/s eta 0:01:37
   ------------------------------ ------

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

#Define the class names
class_names=["Bird-drop","Clean","Dusty","Electrical-damage","Physical-damage","Snow-covered"]

#save class names to a file
with open('labels.txt','w')as f:
    for class_name in class_names:
        f.write(f"{class_name}\n")

#define the image size and batch size
img_height,img_width=150,150
batch_size=32

#Directory paths for training and validation datasets
train_dir=r'E:\itp\solar panel dust detection\Train_set'
val_dir=r'E:\itp\solar panel dust detection\Test_set'

#Image data generators for augmenting the training and validation data
train_datagen=ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen=ImageDataGenerator(rescale=1.0/255.0)

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator=val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

#define the CNN model
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(img_height,img_width,3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(512,activation='relu'),
    Dropout(0.5),
    Dense(len(class_names),activation='softmax')
])

#compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

#train the model
epochs=20
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples //batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples //batch_size,
    epochs=epochs
)

#Save the model to an .h5 file
model.save('Solar_fault_detection_model.h5')

print("Model and Labels saved successfully.")

In [9]:
pip install pyttsx3

   ---------------------------------------- 0.0/219.4 kB ? eta -:--:--
   -------------------- ------------------- 112.6/219.4 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 219.4/219.4 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pyttsx3

# load the trained model
model=tf.keras.models.load_model('solar_fault_detection_model.h5')

#load class names from lables.txt
class_names=[]
with open('labels.txt','r') as f:
    class_names=f.read().splitlines()

#initialize the camera
cap=cv2.VideoCapture(0)

#Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error:could not open camera.")
    exit()

#initialize the text-to-speech engine
engine = pyttsx3.init()

#Define the image size
img_height,img_width=150,150

previous_class_name=""

while True:
    ret,frame=cap.read()
    if not ret:
        break

    #Convert the image to RGB and resize it 
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized_frame=cv2.resize(rgb_frame,(img_height,img_width))

    #Normalize the image
    normalized_frame=resized_frame /255.0

    #Expand dimensions tp match the input shape of the model
    input_frame=np.expand_dims(normalized_frame,axis=0)

    #Predict the class
    predictions=model.predict(input_frame)
    predicted_class=np.argmax(predictions[0])
    class_name=class_names[predicted_class]

    #If the detected class changes, speak the class name
    if class_name !=previous_class_name:
        engine.say(class_name)
        engine.runAndWait()
        previous_class_name=class_name

    #Draw a bounding box and the class name on the original frame
    height,width, _ = frame.shape
    cv2.rectangle(frame,(10,10),(width - 10,height - 10),(0,255,0),2)
    cv2.putText(frame,class_name,(20,40),cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0), 2, cv2.LINE_AA)

    #Display the resulting frame
    cv2.imshow('Solar Fault Detection',frame)

    #Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#Release the camera and close windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━